In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/GA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1168647203,GA,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,1393637109,GA,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,1814537102,GA,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,1847307110,GA,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,2680678506,GA,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1168647203,GA,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,1393637109,GA,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,1814537102,GA,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,1847307110,GA,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,2680678506,GA,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
302,9734337010,GA,"$3,251,200.00, CONYERS, GA",33.667610,-84.017690,NaN
3681,5698907302,GA,"5855 Oakbrook Pkwy Ste H, NORCROSS, GA",33.915970,-84.196946,NaN
3719,1151418400,GA,"229, CANTON, GA",32.157435,-82.907123,NaN
4009,3646958410,GA,"3535 PEACHTREE RD NE STE 520, ATLANTA, GA",33.851826,-84.358456,NaN
4456,3823027800,GA,"4132 MAPLE LN, POWDER SPGS, GA",33.871938,-84.674248,NaN
6278,5089558305,GA,"0132, CANTON, GA",34.236762,-84.490762,NaN
7582,5679327103,GA,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
9014,3000858209,GA,"124 Rolling Hill Dr, WILLIAMSON, GA",33.192282,-84.339898,NaN
9132,7842647203,GA,"1067 SAND DOLLAR WAY, BRUNSWICK, GA",31.132038,-81.601493,NaN
10194,6065397004,GA,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('13')] #GA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/26 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

26


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11729/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
302,9734337010,GA,"$3,251,200.00, CONYERS, GA",33.667610,-84.017690,[132470603101023]
3681,5698907302,GA,"5855 Oakbrook Pkwy Ste H, NORCROSS, GA",33.915970,-84.196946,[131350504372008]
3719,1151418400,GA,"229, CANTON, GA",32.157435,-82.907123,[130919605001015]
4009,3646958410,GA,"3535 PEACHTREE RD NE STE 520, ATLANTA, GA",33.851826,-84.358456,[131210096073002]
4456,3823027800,GA,"4132 MAPLE LN, POWDER SPGS, GA",33.871938,-84.674248,[130670315061003]
6278,5089558305,GA,"0132, CANTON, GA",34.236762,-84.490762,[130570906041016]
7582,5679327103,GA,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,[]
9014,3000858209,GA,"124 Rolling Hill Dr, WILLIAMSON, GA",33.192282,-84.339898,[132551612011016]
9132,7842647203,GA,"1067 SAND DOLLAR WAY, BRUNSWICK, GA",31.132038,-81.601493,[131270010024048]
10194,6065397004,GA,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11729/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
302,9734337010,GA,"$3,251,200.00, CONYERS, GA",33.667610,-84.017690,132470603101023
3681,5698907302,GA,"5855 Oakbrook Pkwy Ste H, NORCROSS, GA",33.915970,-84.196946,131350504372008
3719,1151418400,GA,"229, CANTON, GA",32.157435,-82.907123,130919605001015
4009,3646958410,GA,"3535 PEACHTREE RD NE STE 520, ATLANTA, GA",33.851826,-84.358456,131210096073002
4456,3823027800,GA,"4132 MAPLE LN, POWDER SPGS, GA",33.871938,-84.674248,130670315061003
6278,5089558305,GA,"0132, CANTON, GA",34.236762,-84.490762,130570906041016
7582,5679327103,GA,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
9014,3000858209,GA,"124 Rolling Hill Dr, WILLIAMSON, GA",33.192282,-84.339898,132551612011016
9132,7842647203,GA,"1067 SAND DOLLAR WAY, BRUNSWICK, GA",31.132038,-81.601493,131270010024048
10194,6065397004,GA,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
7582,5679327103,GA,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
10194,6065397004,GA,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN
12198,9795107701,GA,"21505 Augusta Ave, Port Charlotte, GA",26.989473,-82.092202,NaN
15805,8389928402,GA,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN
16051,8809168603,GA,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN
19521,3528947105,GA,"P O Box 587 1975 N MAIN ST, LA FAYETTE, GA",34.066495,-118.218462,NaN
20169,9526287700,GA,"613 WEST ASHLEY ST, JACKSONVILLE, GA",30.332706,-81.664247,NaN
20990,1077037104,GA,"1504 HWY 195, LEESBURG, GA",30.724764,-97.670598,NaN
24753,8123187107,GA,"1130 ADA ST., STE B, GA",29.381239,-98.455152,NaN
24806,7425547000,GA,"9940 HIGHWAY 82, WOODSTOCK, GA",33.351056,-93.368070,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
302,9734337010,132470603101023
3681,5698907302,131350504372008
3719,1151418400,130919605001015
4009,3646958410,131210096073002
4456,3823027800,130670315061003
6278,5089558305,130570906041016
7582,5679327103,NaN
9014,3000858209,132551612011016
9132,7842647203,131270010024048
10194,6065397004,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/GA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,264220,1168647203,GA,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,264221,1393637109,GA,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,264222,1814537102,GA,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,264223,1847307110,GA,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,264224,2680678506,GA,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,264220,1168647203,GA,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14,NaN
1,264221,1393637109,GA,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14,NaN
2,264222,1814537102,GA,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14,NaN
3,264223,1847307110,GA,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14,NaN
4,264224,2680678506,GA,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14,NaN
...,...,...,...,...,...,...,...,...
25782,290002,9788108002,GA,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,1.306703e+14,NaN
25783,290003,9791658407,GA,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,1.321501e+14,NaN
25784,290004,9855898507,GA,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,1.305709e+14,NaN
25785,290005,9870708407,GA,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,1.305709e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,264220,1168647203,GA,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14,NaN,131210095042002.0
1,264221,1393637109,GA,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14,NaN,131210091033003.0
2,264222,1814537102,GA,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14,NaN,131210114342000.0
3,264223,1847307110,GA,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14,NaN,131210116493006.0
4,264224,2680678506,GA,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14,NaN,130630402042001.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1168647203,GA,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,131210095042002.0
1,1393637109,GA,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,131210091033003.0
2,1814537102,GA,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,131210114342000.0
3,1847307110,GA,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,131210116493006.0
4,2680678506,GA,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,130630402042001.0
...,...,...,...,...,...,...
25782,9788108002,GA,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,130670315133031.0
25783,9791658407,GA,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,132150103032020.0
25784,9855898507,GA,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,130570901013036.0
25785,9870708407,GA,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,130570905042006.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
7582,5679327103,GA,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
10194,6065397004,GA,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN
12198,9795107701,GA,"21505 Augusta Ave, Port Charlotte, GA",26.989473,-82.092202,NaN
15805,8389928402,GA,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN
16051,8809168603,GA,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN
19521,3528947105,GA,"P O Box 587 1975 N MAIN ST, LA FAYETTE, GA",34.066495,-118.218462,NaN
20169,9526287700,GA,"613 WEST ASHLEY ST, JACKSONVILLE, GA",30.332706,-81.664247,NaN
20990,1077037104,GA,"1504 HWY 195, LEESBURG, GA",30.724764,-97.670598,NaN
24753,8123187107,GA,"1130 ADA ST., STE B, GA",29.381239,-98.455152,NaN
24806,7425547000,GA,"9940 HIGHWAY 82, WOODSTOCK, GA",33.351056,-93.368070,NaN


Convert to File

In [19]:
fips_GA = fips_merge_drop

In [20]:
fips_GA.to_csv("../../../data/state_data/geo/geo_fips/23/GA_fips_scraped.csv") 